In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "./data/new_daily"
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df

,code,close_volume_corr,date,open,close,low,high,volume,money,turnover_ratio
0,000001.XSHE,0.126032,2021-01-04,2306.70,2246.31,2226.99,2306.70,1286929.0,2.891682e+09,0.8009
1,000001.XSHE,0.089958,2021-01-05,2222.16,2194.38,2149.70,2231.82,1508123.0,3.284607e+09,0.9386
2,000001.XSHE,-0.216109,2021-01-06,2183.51,2362.25,2173.85,2362.25,1602181.0,3.648522e+09,0.9971
3,000001.XSHE,0.088208,2021-01-07,2357.42,2403.31,2322.40,2412.97,1311744.0,3.111275e+09,0.8163
4,000001.XSHE,0.569952,2021-01-08,2403.31,2397.27,2332.06,2427.47,989881.0,2.348316e+09,0.6160
...,...,...,...,...,...,...,...,...,...,...
1058890,689009.XSHG,0.277105,2021-12-27,65.00,67.08,63.56,68.43,3163564.0,2.086403e+08,0.7141
1058891,689009.XSHG,0.473949,2021-12-28,66.60,68.15,66.16,68.95,3561048.0,2.416030e+08,0.8039
1058892,689009.XSHG,0.520082,2021-12-29,68.50,67.58,66.64,69.68,3363471.0,2.282845e+08,0.7593
1058893,689009.XSHG,0.337465,2021-12-30,67.99,68.24,66.01,68.69,3059763.0,2.060038e+08,0.6907


In [23]:
result_df['Turn20'] = result_df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).mean())
result_df

,code,volume_mean,volume_std,date,open,close,low,high,volume,money,turnover_ratio,Turn20
0,000001.XSHE,39037.834365,24733.158091,2021-01-04,2306.70,2246.31,2226.99,2306.70,1286929.0,2.891682e+09,0.8009,NaN
1,000001.XSHE,45747.580731,25216.049582,2021-01-05,2222.16,2194.38,2149.70,2231.82,1508123.0,3.284607e+09,0.9386,NaN
2,000001.XSHE,48600.739023,26962.238603,2021-01-06,2183.51,2362.25,2173.85,2362.25,1602181.0,3.648522e+09,0.9971,NaN
3,000001.XSHE,39790.573671,30609.861163,2021-01-07,2357.42,2403.31,2322.40,2412.97,1311744.0,3.111275e+09,0.8163,NaN
4,000001.XSHE,30027.147223,19673.819288,2021-01-08,2403.31,2397.27,2332.06,2427.47,989881.0,2.348316e+09,0.6160,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1058890,689009.XSHG,65907.583333,42235.492048,2021-12-27,65.00,67.08,63.56,68.43,3163564.0,2.086403e+08,0.7141,0.876175
1058891,689009.XSHG,74188.500000,36329.116414,2021-12-28,66.60,68.15,66.16,68.95,3561048.0,2.416030e+08,0.8039,0.899040
1058892,689009.XSHG,70072.312500,33892.455355,2021-12-29,68.50,67.58,66.64,69.68,3363471.0,2.282845e+08,0.7593,0.913115
1058893,689009.XSHG,63745.062500,24952.602456,2021-12-30,67.99,68.24,66.01,68.69,3059763.0,2.060038e+08,0.6907,0.920320


In [2]:
df = result_df.copy()
df['tmp1'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=60).sum())
df['tmp2'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).sum())
df['tmp'] = (df['tmp1'] - df['tmp2']) / 40
df['tmp'] = df['turnover_ratio'] / df['tmp'] - 1
df['PctTurn20'] = df.groupby('code')['tmp'].transform(lambda x: x.rolling(window=20).mean())

In [3]:
df

,code,close_volume_corr,date,open,close,low,high,volume,money,turnover_ratio,tmp1,tmp2,tmp,PctTurn20
0,000001.XSHE,0.126032,2021-01-04,2306.70,2246.31,2226.99,2306.70,1286929.0,2.891682e+09,0.8009,NaN,NaN,NaN,NaN
1,000001.XSHE,0.089958,2021-01-05,2222.16,2194.38,2149.70,2231.82,1508123.0,3.284607e+09,0.9386,NaN,NaN,NaN,NaN
2,000001.XSHE,-0.216109,2021-01-06,2183.51,2362.25,2173.85,2362.25,1602181.0,3.648522e+09,0.9971,NaN,NaN,NaN,NaN
3,000001.XSHE,0.088208,2021-01-07,2357.42,2403.31,2322.40,2412.97,1311744.0,3.111275e+09,0.8163,NaN,NaN,NaN,NaN
4,000001.XSHE,0.569952,2021-01-08,2403.31,2397.27,2332.06,2427.47,989881.0,2.348316e+09,0.6160,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058890,689009.XSHG,0.277105,2021-12-27,65.00,67.08,63.56,68.43,3163564.0,2.086403e+08,0.7141,70.7070,17.5235,-0.462916,-0.481452
1058891,689009.XSHG,0.473949,2021-12-28,66.60,68.15,66.16,68.95,3561048.0,2.416030e+08,0.8039,69.4748,17.9808,-0.375539,-0.456996
1058892,689009.XSHG,0.520082,2021-12-29,68.50,67.58,66.64,69.68,3363471.0,2.282845e+08,0.7593,68.6944,18.2623,-0.397765,-0.436729
1058893,689009.XSHG,0.337465,2021-12-30,67.99,68.24,66.01,68.69,3059763.0,2.060038e+08,0.6907,67.5511,18.4064,-0.437823,-0.420181


In [4]:
df = result_df.copy()
df['STR'] = df.groupby('code')['turnover_ratio'].transform(lambda x: x.rolling(window=20).std())

In [5]:
#将日期列转换为datetime类型
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='./newclass_feature/STR'
selected_columns=['code', 'STR']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [22]:
#data
path = './new_factor_neutralization/UTR_1'
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df

,code,Turn20,STR,date
0,000001.XSHE,NaN,NaN,2020-01-02
1,000001.XSHE,NaN,NaN,2020-01-03
2,000001.XSHE,NaN,NaN,2020-01-06
3,000001.XSHE,NaN,NaN,2020-01-07
4,000001.XSHE,NaN,NaN,2020-01-08
...,...,...,...,...
4253855,689009.XSHG,0.004146,0.024589,2023-11-24
4253856,689009.XSHG,0.004400,0.024345,2023-11-27
4253857,689009.XSHG,0.004508,0.021538,2023-11-28
4253858,689009.XSHG,0.004490,0.018567,2023-11-29


In [24]:
df = result_df.copy()
df['UTR2'] = df['STR'] + (df['STR'] / (1+np.abs(df['STR']))) * df['Turn20']
df

,code,Turn20,STR,date,UTR2
0,000001.XSHE,NaN,NaN,2020-01-02,NaN
1,000001.XSHE,NaN,NaN,2020-01-03,NaN
2,000001.XSHE,NaN,NaN,2020-01-06,NaN
3,000001.XSHE,NaN,NaN,2020-01-07,NaN
4,000001.XSHE,NaN,NaN,2020-01-08,NaN
...,...,...,...,...,...
4253855,689009.XSHG,0.004146,0.024589,2023-11-24,0.024689
4253856,689009.XSHG,0.004400,0.024345,2023-11-27,0.024449
4253857,689009.XSHG,0.004508,0.021538,2023-11-28,0.021633
4253858,689009.XSHG,0.004490,0.018567,2023-11-29,0.018649


In [27]:
df = result_df.copy()
df['score1'] = df.groupby('date')['STR'].transform(lambda x: x.rank())
df['tag1'] = df.groupby('date')['STR'].transform(lambda x: x.rank(pct=True))
df['score2'] = 0
tmp1 = df[df['tag1'] < 0.5]
tmp1['score2'] = tmp1.groupby('date')['Turn20'].transform(lambda x: x.rank(ascending=False))
tmp2 = df[df['tag1'] >= 0.5]
tmp2['score2'] = tmp2.groupby('date')['Turn20'].transform(lambda x: x.rank())
final = pd.concat([tmp1, tmp2]).sort_values(by=['date', 'code'])
final = final.reset_index(drop=True)
final['UTR'] = (final['score1'] + final['score2'])


In [28]:
final

,code,Turn20,STR,date,score1,tag1,score2,UTR
0,000001.XSHE,0.018561,0.088041,2020-02-06,2770.0,0.735724,926.0,3696.0
1,000002.XSHE,0.021163,0.092927,2020-02-06,2801.0,0.743958,1024.0,3825.0
2,000004.XSHE,-0.005481,-0.056563,2020-02-06,1293.0,0.343426,362.0,1655.0
3,000005.XSHE,-0.022760,-0.125835,2020-02-06,426.0,0.113147,1419.0,1845.0
4,000006.XSHE,-0.015139,-0.092808,2020-02-06,802.0,0.213015,945.0,1747.0
...,...,...,...,...,...,...,...,...
4182640,688799.XSHG,-0.009183,-0.081756,2023-11-30,1687.0,0.332087,563.0,2250.0
4182641,688800.XSHG,-0.001277,0.027055,2023-11-30,3310.0,0.651575,343.0,3653.0
4182642,688819.XSHG,0.010952,0.026382,2023-11-30,3300.0,0.649606,1117.0,4417.0
4182643,688981.XSHG,0.027255,0.134337,2023-11-30,4223.0,0.831299,1764.0,5987.0


In [29]:
#将日期列转换为datetime类型
df = final
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='./new_feature/UTR_final'
selected_columns=['code', 'UTR']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [30]:
#将日期列转换为datetime类型
# df = final
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='./new_feature/UTR_1'
selected_columns=['code', 'Turn20', 'STR']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [31]:
import statsmodels.api as sm
results = sm.OLS(final['UTR'], final[['STR', 'Turn20']]).fit()    
final['UTR_resid']=results.resid    #取残差项作为中性化后的因子值
final

,code,Turn20,STR,date,score1,tag1,score2,UTR,UTR_resid
0,000001.XSHE,0.018561,0.088041,2020-02-06,2770.0,0.735724,926.0,3696.0,2581.327677
1,000002.XSHE,0.021163,0.092927,2020-02-06,2801.0,0.743958,1024.0,3825.0,2571.811953
2,000004.XSHE,-0.005481,-0.056563,2020-02-06,1293.0,0.343426,362.0,1655.0,2057.025749
3,000005.XSHE,-0.022760,-0.125835,2020-02-06,426.0,0.113147,1419.0,1845.0,3254.454632
4,000006.XSHE,-0.015139,-0.092808,2020-02-06,802.0,0.213015,945.0,1747.0,2706.223220
...,...,...,...,...,...,...,...,...,...
4182640,688799.XSHG,-0.009183,-0.081756,2023-11-30,1687.0,0.332087,563.0,2250.0,2892.537434
4182641,688800.XSHG,-0.001277,0.027055,2023-11-30,3310.0,0.651575,343.0,3653.0,3650.793655
4182642,688819.XSHG,0.010952,0.026382,2023-11-30,3300.0,0.649606,1117.0,4417.0,3820.193507
4182643,688981.XSHG,0.027255,0.134337,2023-11-30,4223.0,0.831299,1764.0,5987.0,4338.139298


In [32]:
#将日期列转换为datetime类型
# df = final
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='./new_feature/UTR_final2'
selected_columns=['code', 'UTR_resid']

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [26]:
#homework1
path = './new_factor_neutralization/UTR_1'
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    date = os.path.basename(filename)[:-4]
    df['date'] = date
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df
df = result_df.copy()
df['UTR2'] = df['STR'] + (df['STR'] / (1+np.abs(df['STR']))) * df['Turn20']
df['UTR3'] = df['STR'] + ((np.abs(df['STR']) + df['STR']) / 2) * df['Turn20']  #relu


,code,Turn20,STR,date,UTR2,UTR3
0,000001.XSHE,NaN,NaN,2020-01-02,NaN,NaN
1,000001.XSHE,NaN,NaN,2020-01-03,NaN,NaN
2,000001.XSHE,NaN,NaN,2020-01-06,NaN,NaN
3,000001.XSHE,NaN,NaN,2020-01-07,NaN,NaN
4,000001.XSHE,NaN,NaN,2020-01-08,NaN,NaN
...,...,...,...,...,...,...
4253855,689009.XSHG,0.004146,0.024589,2023-11-24,0.024689,0.024691
4253856,689009.XSHG,0.004400,0.024345,2023-11-27,0.024449,0.024452
4253857,689009.XSHG,0.004508,0.021538,2023-11-28,0.021633,0.021636
4253858,689009.XSHG,0.004490,0.018567,2023-11-29,0.018649,0.018650


In [ ]:
import statsmodels.api as sm
result2 = sm.OLS(df['UTR2'], df[['STR', 'Turn20']]).fit()    
df['UTR2_resid']=result2.resid    #取残差项作为中性化后的因子值
result3 = sm.OLS(df['UTR3'], df[['STR', 'Turn20']]).fit()    
df['UTR3_resid']=result3.resid    #取残差项作为中性化后的因子值
df

In [27]:
#homework2
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = "./data/data_daily"
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
result_df['vwap'] = result_df['money'] / result_df['volume']
result_df

,date,code,open,close,low,high,volume,money,turnover_ratio,vwap
0,2020-01-02,000001.XSHE,1976.78,2002.90,1964.91,2012.40,1288881.0,2.571196e+09,0.7885,1994.906033
1,2020-01-03,000001.XSHE,2011.21,2039.71,2008.84,2055.14,940146.0,1.914495e+09,0.5752,2036.381024
2,2020-01-06,000001.XSHE,2019.52,2026.65,2007.65,2058.70,726114.0,1.477930e+09,0.4442,2035.396912
3,2020-01-07,000001.XSHE,2033.77,2036.14,2012.40,2051.58,613690.0,1.247047e+09,0.3755,2032.047345
4,2020-01-08,000001.XSHE,2018.34,1977.97,1974.41,2024.27,714104.0,1.423609e+09,0.4369,1993.559497
...,...,...,...,...,...,...,...,...,...,...
4253855,2023-11-24,689009.XSHG,34.88,35.28,34.88,35.88,10815571.0,3.834648e+08,2.0661,35.454882
4253856,2023-11-27,689009.XSHG,35.24,34.89,34.62,35.60,6348515.0,2.225542e+08,1.2128,35.056107
4253857,2023-11-28,689009.XSHG,34.90,34.36,34.01,35.17,5755962.0,1.979950e+08,1.0996,34.398252
4253858,2023-11-29,689009.XSHG,34.59,33.27,33.09,34.59,6423819.0,2.155980e+08,1.2271,33.562281


In [28]:
ziduan_list = ['open','close','low','high','volume','money','turnover_ratio']

for ziduan in tqdm(ziduan_list):
    exec(f"{ziduan} = result_df.pivot(index='date', columns='code', values='{ziduan}')")
    exec(f"{ziduan}.to_pickle('./data/data_daily_pkl/{ziduan}.pkl')")
def ts_mean(df):
    return df.rolling(20).mean()


100%|█████████████████████████████████████████████| 7/7 [00:08<00:00,  1.28s/it]


In [29]:
%time
eval('ts_mean(open)')

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 6.91 µs


code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688787.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688798.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-24,1316.5740,1984.9690,132.8130,14.4545,236.6640,60.7640,64.1005,111.2000,50.4000,41.6910,...,103.7050,56.4930,147.8620,54.0545,114.5215,47.2945,60.3130,33.3495,55.6170,33.7140
2023-11-27,1314.6070,1989.1905,133.3270,14.4700,237.8255,60.8300,64.2350,111.9015,50.2070,41.7910,...,104.1730,56.5650,147.6010,54.2790,114.8755,47.3470,60.7210,33.2520,55.4845,33.8455
2023-11-28,1311.8145,1991.2170,133.6210,14.4595,238.7550,60.8540,64.3560,112.5425,49.8480,41.9305,...,104.5705,56.5600,147.4230,54.4510,114.4485,47.2035,61.0020,33.1625,55.3000,33.9355


In [30]:
%time
result_df['test'] = result_df.groupby('code')['open'].transform(lambda x: x.rolling(window=20).mean())

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 16.9 µs


In [7]:
import pandas as pd
df = pd.read_csv('final_barra.csv')
df

,Unnamed: 0.1,Unnamed: 0,code,date,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage
0,0,0,000001.XSHE,2023-12-01,1.068393,-0.181433,-0.677871,-0.616431,-1.253025,2.635215,0.025190,1.716058,0.304432,1.941336
1,1,1,000002.XSHE,2023-12-01,1.068393,0.301413,-1.215908,-0.102159,-1.253025,2.635215,0.656534,1.645083,-0.293361,1.993427
2,2,2,000004.XSHE,2023-12-01,-2.794197,-0.659882,1.085002,2.743066,-0.172072,-1.528734,1.624167,-2.903963,-2.897109,-0.928984
3,3,3,000005.XSHE,2023-12-01,-3.292658,-1.050067,-0.491306,0.788779,-1.671914,0.772022,-1.609956,-2.166244,-1.753739,-1.052188
4,4,4,000006.XSHE,2023-12-01,-1.758643,-0.734535,-0.130651,1.846150,2.023071,2.203469,0.722483,0.269346,-0.406998,1.850387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581619,53355,53355,688799.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581620,53356,53356,688800.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581621,53357,53357,688819.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581622,53358,53358,688981.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df

,code,date,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage
0,000001.XSHE,2023-12-01,1.068393,-0.181433,-0.677871,-0.616431,-1.253025,2.635215,0.025190,1.716058,0.304432,1.941336
1,000002.XSHE,2023-12-01,1.068393,0.301413,-1.215908,-0.102159,-1.253025,2.635215,0.656534,1.645083,-0.293361,1.993427
2,000004.XSHE,2023-12-01,-2.794197,-0.659882,1.085002,2.743066,-0.172072,-1.528734,1.624167,-2.903963,-2.897109,-0.928984
3,000005.XSHE,2023-12-01,-3.292658,-1.050067,-0.491306,0.788779,-1.671914,0.772022,-1.609956,-2.166244,-1.753739,-1.052188
4,000006.XSHE,2023-12-01,-1.758643,-0.734535,-0.130651,1.846150,2.023071,2.203469,0.722483,0.269346,-0.406998,1.850387
...,...,...,...,...,...,...,...,...,...,...,...,...
581619,688799.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581620,688800.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581621,688819.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581622,688981.XSHG,2024-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.columns

Index(['code', 'date', 'size', 'beta', 'momentum', 'residual_volatility',
       'non_linear_size', 'book_to_price_ratio', 'liquidity', 'earnings_yield',
       'growth', 'leverage'],
      dtype='object')

In [10]:
import os
df['date']=pd.to_datetime(df['date'])

#保存文件夹路径
output_folder='./data_barra_24'
selected_columns=df.columns

os.makedirs(output_folder,exist_ok=True)

#创建日期索引并检查是否在dateframe中存在
date_index=pd.date_range(df['date'].min(),df['date'].max(),freq='D')
existing_dates=[d for d in date_index if d in df['date'].values]

#遍历每个日期如果它在dataframe中存在 则将其保存为单独的csv文件
for date in existing_dates:
    group=df.loc[df['date']==date,selected_columns]
    filename=os.path.join(output_folder,f'{date.strftime("%Y-%m-%d")}.csv')
    group=group.sort_values(by='code',ascending=True)
    group.to_csv(filename,index=False)

In [11]:
#homework2
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import glob
import warnings
warnings.filterwarnings('ignore')


#data
path = './data_barra_24'
all_files = glob.glob(path + "/*.csv")   #把所有的数据文件文件名读取在一起

li = []


for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)


frame = pd.concat(li, axis=0, ignore_index=True)
result_df = frame.sort_values(by=['code', 'date'], ascending=True)
result_df = result_df.reset_index(drop=True)
# result_df['vwap'] = result_df['money'] / result_df['volume']
result_df

,code,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage,date
0,000001.XSHE,1.068393,-0.181433,-0.677871,-0.616431,-1.253025,2.635215,0.025190,1.716058,0.304432,1.941336,2023-12-01
1,000001.XSHE,1.071782,-0.180434,-0.726913,-0.621268,-1.255759,2.637929,0.014810,1.715323,0.305574,1.942164,2023-12-04
2,000001.XSHE,1.071757,-0.188137,-0.699693,-0.596433,-1.255506,2.635112,0.023442,1.711017,0.304550,1.939162,2023-12-05
3,000001.XSHE,1.075320,-0.189296,-0.744442,-0.606806,-1.257455,2.641648,0.029528,1.709784,0.302144,1.943025,2023-12-06
4,000001.XSHE,1.074259,-0.189439,-0.751426,-0.612921,-1.258675,2.635173,0.015030,1.714652,0.302506,1.941687,2023-12-07
...,...,...,...,...,...,...,...,...,...,...,...,...
4849031,689009.XSHG,-0.490939,0.148068,-1.563567,1.084458,1.102396,-1.057590,-0.552306,1.949907,-0.603977,-0.673255,NaN
4849032,689009.XSHG,-0.335042,-0.456570,-1.127183,0.514389,0.874563,-1.318404,-1.058902,1.931653,-0.508999,-0.850862,NaN
4849033,689009.XSHG,-0.446331,0.600112,-0.979857,0.547492,1.048255,-1.190129,-0.324947,1.856775,-0.281218,-0.764579,NaN
4849034,689009.XSHG,-0.129605,-0.490968,-0.211745,0.661072,0.510825,-1.314173,-0.582296,1.963177,-0.464037,-0.801399,NaN


In [16]:
result_df[result_df['date'] == '2024-05-17']

,code,size,beta,momentum,residual_volatility,non_linear_size,book_to_price_ratio,liquidity,earnings_yield,growth,leverage,date
108,000001.XSHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
1166,000002.XSHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
2224,000004.XSHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
3282,000005.XSHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
4340,000006.XSHE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...
4845177,688799.XSHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
4845866,688800.XSHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
4846548,688819.XSHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
4847353,688981.XSHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17
